In [1]:
#LangChain에서 지원하는 Vector Store
#=> https://python.langchain.com/v0.1/docs/integrations/vectorstores/

#Chroma
# 크로마는 오픈소스 벡터 데이터베이스입니다. Chroma는 Apache 2.0 라이선스가 부여됩니다.
# => 공식도큐먼트 : https://docs.trychroma.com/
# => 홈페이지 :  https://www.trychroma.com/

# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")

#%pip install -qU chromadb


In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("../data/04.급여규정_16.06.01.txt") as f:
    file = f.read()
print(f'*type:{type(file)}')


text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

# text_splitter 이용해서 file 텍스트를 문서단위로 출력
# -> separator를 지정하지 않았으므로, 문서에서 기본값 "\n\n" 로 구분해서 나눔.
split_docs = text_splitter.create_documents([file])
print(f'*문서 분할 수: {len(split_docs)}')
for i, split_doc in enumerate(split_docs):
    print(f"Doc {i} : {split_doc}\n")

*type:<class 'str'>
*문서 분할 수: 17
Doc 0 : page_content='급여규정\n1. 목적\n이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.\n2. 적용범위\n직원의 급여는 관계법령, 기타 별도로 정한 것을 제외하고는 이 규정이 정하는 바에 따른다.\n3. 급여의 구성\n3.1 급여는 기본급, 제수당, 상여금, 퇴직금으로 구분한다.\n3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정'

Doc 1 : page_content='3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.\n5. 계산과 지급\n5.1 계산기간\n급여계산기간은 매월 1일부터 당월 말일까지로 한다.\n5.2 일할계산\n5.2.1 급여의 변경이 있는 월의 급여는 일할계산한다.\n5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수'

Doc 2 : page_content='5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수\n일할계산에 있어 그 일할일수는 실근로일수를 기준으로 한다.\n5.4 계산단위\n급여계산은 원단위에서 이를 절사한다.\n5.5 급여지급일\n5.5.1 급여지급일은 매월 20일로 한다.\n단, 지급일이 휴일인 경우에는 그 전일로 한다.\n5.5.2 비상재해, 기타 부득이한 사유가 발생시에는 지급일을 변경할 수 있다\n5.6 지급방법'

Doc 3 : page_content='5.6 지급방법\n급여는 통화로 전액을 본인에게 직접 지급하거나 본인이 신청한 본인의 실명계좌에 지급한다.\n5.7 신규채용 및 복직자 급여\n5.7.1 신규채용 및 복직자의 급여는 발령일로부터 일할계산한다.\n5.7.2 신규채용되어 수습기간중에 있는 자는 월급여의 90%를 지급한다.\n5.7.3 복직

In [4]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# 오픈 소스 임베딩 함수를 생성합니다.
#stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
stf_embeddings = SentenceTransformerEmbeddings(model_name="bongsoo/kpf-sbert-128d-v1")

text = "임베딩 테스트를 하기 위한 샘플 문장입니다."  # 테스트용 문서 텍스트를 정의합니다.
query_result = stf_embeddings.embed_query(text)
print(f'*벡터길이:{len(query_result)}')


*벡터길이:128


In [5]:
from langchain_community.vectorstores import Chroma

# Chroma에 로드합니다.
%time db = Chroma.from_documents(split_docs, stf_embeddings)

CPU times: user 16.4 s, sys: 287 ms, total: 16.7 s
Wall time: 2.65 s


In [6]:
# 질의합니다.
query = "신규 입사자 급여는 얼마?"
similar_docs = db.similarity_search(query)
print(f"*검색된 docs 계수: {len(similar_docs)}")

for i, similar_doc in enumerate(similar_docs):
    print(f"Doc {i} : {similar_doc}\n")


*검색된 docs 계수: 4
Doc 0 : page_content='7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'

Doc 1 : page_content='10. 부록\n10.1시행일\n- 이 규정은 2007년 9월 14일부터 제정·시행한다.\n- 이 규정은 2016년 6월 1일부터 개정, 시행한다.\n급여규정 1/3'

Doc 2 : page_content='을 공제한 금액으로 한다.\n5.8.3 징집명령에 의한 휴직자에 대하여는 휴직발령일로부터 복직발령일 전일까지 급여를 지급하지 \n아니한다.\n5.8.4 기타 개인적인 사정으로 휴직하였을 경우 그 기간동안의 급여는 지급하지 아니한다.\n5.9 퇴직자 급여\n5.9.1 퇴직자는 퇴직명령전일까지의 급여를 일할계산한다. .\n5.9.2 급여 지급일 전 최소 1개월 이전에 퇴직의사를 밝히지 않아 발생한 퇴직 당월의 과지급'

Doc 3 : page_content='6.6.1 직원이 상위직급으로 승진한 경우 승진된 직급의 초임을 적용함을 원칙으로 한다.\n6.6.2 승진후 월급여가 승진전 월급여보다 적은 경우에는 월급여는 승진된 직급의 초임을 적용하되 \n그 차액은 조정수당으로 지급한다. \n단, 임금인상, 승급 등에 의한 그 차액의 감소분만큼은 조정수당을 차감, 조정한다.\n7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금'



In [8]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
db2 = Chroma.from_documents(split_docs, stf_embeddings, persist_directory=DB_PATH)

# 질의합니다.
query = "신규 입사자 급여는 얼마?"
similar_docs = db2.similarity_search(query)
print(f"*검색된 docs 계수: {len(similar_docs)}")

for i, similar_doc in enumerate(similar_docs):
    print(f"Doc {i} : {similar_doc}\n")


*검색된 docs 계수: 4
Doc 0 : page_content='7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'

Doc 1 : page_content='10. 부록\n10.1시행일\n- 이 규정은 2007년 9월 14일부터 제정·시행한다.\n- 이 규정은 2016년 6월 1일부터 개정, 시행한다.\n급여규정 1/3'

Doc 2 : page_content='을 공제한 금액으로 한다.\n5.8.3 징집명령에 의한 휴직자에 대하여는 휴직발령일로부터 복직발령일 전일까지 급여를 지급하지 \n아니한다.\n5.8.4 기타 개인적인 사정으로 휴직하였을 경우 그 기간동안의 급여는 지급하지 아니한다.\n5.9 퇴직자 급여\n5.9.1 퇴직자는 퇴직명령전일까지의 급여를 일할계산한다. .\n5.9.2 급여 지급일 전 최소 1개월 이전에 퇴직의사를 밝히지 않아 발생한 퇴직 당월의 과지급'

Doc 3 : page_content='6.6.1 직원이 상위직급으로 승진한 경우 승진된 직급의 초임을 적용함을 원칙으로 한다.\n6.6.2 승진후 월급여가 승진전 월급여보다 적은 경우에는 월급여는 승진된 직급의 초임을 적용하되 \n그 차액은 조정수당으로 지급한다. \n단, 임금인상, 승급 등에 의한 그 차액의 감소분만큼은 조정수당을 차감, 조정한다.\n7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금'



In [9]:
# 로컬 디스크에 저장된 Chroma 데이터베이스를 로드 한 뒤 쿼리합니다.
# 디스크에서 문서를 로드합니다.
db3 = Chroma(persist_directory="./chroma_db", embedding_function=stf_embeddings)

# 질의합니다.
query = "신규 입사자 급여는 얼마?"
similar_docs = db2.similarity_search(query)
print(f"*검색된 docs 계수: {len(similar_docs)}")

for i, similar_doc in enumerate(similar_docs):
    print(f"Doc {i} : {similar_doc}\n")


*검색된 docs 계수: 4
Doc 0 : page_content='7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'

Doc 1 : page_content='10. 부록\n10.1시행일\n- 이 규정은 2007년 9월 14일부터 제정·시행한다.\n- 이 규정은 2016년 6월 1일부터 개정, 시행한다.\n급여규정 1/3'

Doc 2 : page_content='을 공제한 금액으로 한다.\n5.8.3 징집명령에 의한 휴직자에 대하여는 휴직발령일로부터 복직발령일 전일까지 급여를 지급하지 \n아니한다.\n5.8.4 기타 개인적인 사정으로 휴직하였을 경우 그 기간동안의 급여는 지급하지 아니한다.\n5.9 퇴직자 급여\n5.9.1 퇴직자는 퇴직명령전일까지의 급여를 일할계산한다. .\n5.9.2 급여 지급일 전 최소 1개월 이전에 퇴직의사를 밝히지 않아 발생한 퇴직 당월의 과지급'

Doc 3 : page_content='6.6.1 직원이 상위직급으로 승진한 경우 승진된 직급의 초임을 적용함을 원칙으로 한다.\n6.6.2 승진후 월급여가 승진전 월급여보다 적은 경우에는 월급여는 승진된 직급의 초임을 적용하되 \n그 차액은 조정수당으로 지급한다. \n단, 임금인상, 승급 등에 의한 그 차액의 감소분만큼은 조정수당을 차감, 조정한다.\n7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금'



In [18]:
# 업데이트 & 삭제

# 간단한 ID 생성
ids = [str(i) for i in range(1, len(split_docs) + 1)]
print(f'*ids:{ids}')

# 데이터 추가
example_db = Chroma.from_documents(split_docs, stf_embeddings, ids=ids)
docs = example_db.similarity_search(query)
print(docs[0].metadata)

# 문서의 메타데이터 업데이트
docs[0].metadata = {
    "source": "급여규정.txt",
    "new_value": "테스트용으로 업데이트할 내용입니다.",
}

# DB 에 업데이트
example_db.update_document(ids[0], docs[0])
print(example_db._collection.get(ids=[ids[0]]))

# 문서 개수 출력
print("*count before", example_db._collection.count())

# 마지막 문서 삭제
example_db._collection.delete(ids=[ids[-1]])

# 삭제 후 문서 개수 출력
print("*count after", example_db._collection.count())


*ids:['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']
{}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'new_value': '테스트용으로 업데이트할 내용입니다.', 'source': '급여규정.txt'}], 'documents': ['7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'], 'uris': None, 'data': None}
*count before 34
*count after 33


In [23]:
# metadatas 중 source 기준으로 필터링
filter = example_db.get(where={"source": "급여규정.txt"})

print(f'*filter type: {type(filter)}')
print(f'{filter}')
print(f'*ids: {filter["ids"]}')

*filter type: <class 'dict'>
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'new_value': '테스트용으로 업데이트할 내용입니다.', 'source': '급여규정.txt'}], 'documents': ['7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'], 'uris': None, 'data': None}
*ids: ['1']
